## 閾値の最適化
Nerder-Meadを使用する例

In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from scipy.optimize import minimize

rand = np.random.RandomState(seed=71)
train_y_prob = np.linspace(0, 1.0, 10000)

train_y = pd.Series(rand.uniform(0.0, 1.0, train_y_prob.size) < train_y_prob)
train_pred_prob = np.clip(train_y_prob * np.exp(rand.standard_normal(train_y_prob.shape) * 0.3), 0.0, 1.0)

init_threshold = 0.5
init_score = f1_score(train_y, train_pred_prob >= init_threshold)
print(init_threshold, init_score)

result = minimize(lambda x: -f1_score(train_y, train_pred_prob >= x), 
                  x0=np.array([0.5]), 
                  method="Nelder-Mead")
best_threshold = result['x'].item()
best_score = f1_score(train_y, train_pred_prob >= best_threshold)
print(best_threshold, best_score)

0.5 0.7224831529507862
0.32324218749999983 0.7557317703844165


## out-of-fold

In [17]:
from sklearn.model_selection import KFold

thresholds = []
scores_tr = []
scores_va = []

kf = KFold(n_splits=4, random_state=71, shuffle=True)
for i, (tr_idx, va_idx) in enumerate(kf.split(train_pred_prob)):
    tr_pred_prob, va_pred_prob = train_pred_prob[tr_idx], train_pred_prob[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    result = minimize(lambda x: -f1_score(tr_y, tr_pred_prob >= x), 
                  x0=np.array([0.5]), 
                  method="Nelder-Mead")
    threshold = result['x'].item()
    score_tr = f1_score(tr_y, tr_pred_prob >= threshold)
    score_va = f1_score(va_y, va_pred_prob >= threshold)
    print(threshold, score_tr, score_va)
    
    thresholds.append(threshold)
    scores_tr.append(score_tr)
    scores_va.append(score_va)
    
threshold_test = np.mean(thresholds)
print(threshold_test)

0.34257812499999984 0.7559183673469387 0.7570422535211268
0.34277343749999983 0.7598457403295548 0.7450980392156863
0.31787109374999983 0.7548253676470588 0.7584803256445047
0.3234374999999998 0.7545569184913447 0.7588603196664351
0.33166503906249983
